In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import solve_bvp

Helper Functions

In [ ]:
total_eps = 0
def eps_tracker(eps):
    total_eps += eps
    return total_eps

sleep_tracker = []
def parent_sleep_tracker(val):
    sleep_tracker.append(val)
    return sleep_tracker

def A(t, w):
    return delta(t)*B(t)*((1/Np(t))*Dp(t))**2
    
def B(t):
    values = np.random.uniform(0, 1, size=1000)
    new_eps = values * np.random.choice([-1, 1], size=1000)
    eps = eps_tracker(new_eps)

    t_adj = t - eps
    for start in range(0, 24, 4):
        if start <= t_adj < start + 3:
            parent_sleep_tracker(1)
            return 1  # Baby asleep
        elif start + 3 <= t_adj < start + 4:
            parent_sleep_tracker(0)
            return 0  # Baby awake
        
    return False  # Case not handled by the current logic

def delta(t):
    if 8 <= t <= 24: # Daytime hours
        return 0.1
    elif 0 < t < 8: # Nighttime hours
        return 0.9
    
    return False # Case not handled by the current logic

# Length of current nap at time t
def Dp(t):
    if sleep_tracker[i] == 0:
        return 0
    
    duration = 0
    for i, sleep in enumerate(sleep_tracker[:t].reverse()):
        if sleep_tracker[i] == 0:
            break
        duration += 1
    return duration

# Number of naps up to time t
def Np(t):
    num_naps = 0
    for i in range(len(sleep_tracker[:t])):
        if sleep_tracker[i] == 1 and (i == 0 or sleep_tracker[i-1] == 0):
            num_naps += 1
    return num_naps

def w(t):
    pass

def u():
    pass

Cost Functional

In [ ]:
def functional(t, y, w):
    # THIS NEEDS TO BE MODIFIED TO WORK WITH SOLVE_BVP
    return x(t, w)

def bc():
    '''
    Boundary Conditions:
    X(t0) = X(tf) = a
    '''
    pass

Solve the ODE

In [ ]:
t0, tf = 0, 24

t_span = (t0, tf)
t = np.linspace(t0, tf, 200)
sol = solve_bvp(functional, t_span, y0, t_eval=t, args=())